# Microstructure Tutorial

## Acknowledgments

Contributions from: Micah Johnson, Mike Durand, HP Marshall, Tate Meehan, Megan Mason, Scott Henderson. This relies heavily on the database and example scripts Micah Johnson created.

## Caveats

The integrating sphere and the SMP data are published at NSIDC; you can read the pages there for documentation etc. However the microCT data are not yet published; please contact Lauren Farnsworth (lauren.b.farnsworth@usace.army.mil) for questions on the CT data.


## Fun Facts About Snow Microstructure

Snow microstructure plays a super important role in snow physics and snow remote sensing, so a lot of effort went towards measuring it in SnowEx 2020!

There are several different quantities that are used to measure snow microstructure, including "grain size". Grain size measurements are challenging to make in a repeatable way, and are also challenging to relate to the physical quantities that control remote sensing measurements. In the last ~15 years or so, a lot of effort has gone into more objective ways to measure microstructure.

Snow microstructure  governs response of remote sensing to snow cover for visible, near-infrared and high-frequency microwave wavelengths. See Figure 1, below, and read {cite:p}`Dozier2009`, for more information.

![dozier-figure](images/dozier2009_fig2.jpg)


<b>Figure 1: Snow microstructure governs visible and near-infrared reflectance. This is figure 2 from {cite:p}`Dozier2009`</b>

Radar measurements such as those made by the Ku-band SWEARR instrument are also very sensitive to snow microstructure. 

![radar-figure](images/modeled-swe-L-response.png)

<b>Figure 2: Modeled response of radar backscatter to SWE and single-scatter abledo (which in turn is a function of snow microstructure), based on a simple model suggested by {cite:p}`Ulaby2014`</b>

As you might imagine, then, snow microstructure is very important to efforts to launch a Ku-band SAR to measure global snow water equivalent (SWE). An important area of research right now is exploring how to use estimates of microstructure (e.g. from snowpack evolution models) to improve SWE retrievals.

Snow microstructure is controlled by other snow properties, such as snow temperature, snow height and snow liquid water content. A really great resource on snow microstructure is Kelly Elder's recent talks:
* [Snow Metamorphism](https://boisestate.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=9a051fe4-db09-4f1d-92db-acd1003cc5cc)
* [Snow Grain Identification](https://boisestate.hosted.panopto.com/Panopto/Pages/Embed.aspx?id=136f7580-c1e2-4c55-ab33-acca003c23d6)

## SnowEx Microstructure Measurement Background

### Basic Microstructure Definitions

Microstructure definitions take a bit of getting used to. It's very easy to get confused. Specific surface area (SSA) is one of the most important quantity used to measure snow microstructure, so that's the focus here. Note that SSA is not the be-all and end-all, so there's a short table describing how to relate SSA to other quantities just below.  A couple of good reads on all of this is {cite:p}`Matzler2002` and {cite:p}`Matzl2010`.


![lowe-figure](images/lowe_etal_2011.png)

<b>Figure 3: Coarse and fine snow microstructure revealed by microCT. This is Figure 2 from {cite:p}`Lowe2011`. Fine-grained snow (a) has high SSA, whereas coarse-grained snow (b) has low SSA.</b>

Use Figure 3 above to ground these definitions: SSA is the surface area of the ice-air interface, normalized in some way. Confusingly, SSA is defined in a couple of different ways in the literature: perhaps ost often, surface area within a particular volume of interest (VOI) is normalized by the mass of the ice in the VOI. As defined in this way, SSA has units of length squared per mass, usually expressed as m<sup>2</sup>/kg. Instead of normalizing by mass, SSA is sometimes defined by normalizing by the volume of the VOI (this is SSA<sub>v</sub> in {cite:p}`Matzl2010`), and sometimes by normalizing by the volume of the ice in the VOI (this is SSA<sub>i</sub> in {cite:p}`Matzl2010`, and q in {cite:p}`Matzler2002`). Here let's just go with the first definition I mentioned:

$$
SSA = \frac{\text{Surface area of ice-air interface}}{\text{Mass of ice}} \quad
$$ 

SSA tends to take values between 5 and 150 m<sup>2</sup>/kg: fresh, fine-grained snow has high SSA, and coarse snow has low SSA. Because it takes a little while for SSA values to become intuitive, a useful derived metric is the equivalent grain diameter (D<sub>eq</sub>; note that this is identical to D<sub>q</sub> in {cite:p}`Matzler2002`), which by definition is the diameter that a sphere would have if it had a particular value of SSA. This is a one-to-one relationship, so there are no assumptions involved.

$$
D_{eq} = \frac{6}{SSA \rho_i} \quad
$$ 

Relationships of specific surface area to other metrics are given in this list if you're curious but otherwise just skip past this bit
* Sometimes people refer to the "optical grain diameter", which is the same as D<sub>eq</sub>. The "optical" refers to {cite:p}`Grenfell1999`, who showed that any snow with a particular SSA had similar (not identical) radiative transfer properties regardless of particle shape in the visible and near-infrared parts of the spectrum. But note the same is not true in the microwave spectrum. 
* Autocorrelation length is usually one of two metrics that summarize the two-point microstructure autocorelation function of the three-dimensional ice-air matrix. Think of the probability that you change media (from ice to air or vice versa) as you move a certain distance within the snow microstructure. The length that defines the likelihood that you'll change media is (an approximation of the correlation length). SSA is by definition (with almost no assumptions) equal to the slope of the autocorrelation function at the origin. But microwave scattering is controlled by correlations at longer lags. For more check out {cite:p}`Matzler2002`. The lack of closing the loop between SSA and correlation length is a significant issue when we have measurements of SSA and microwaves as we do in SnowEx.
* Geometric grain size is what we usually measure when we measure with a hand lens. You can try to relate it to SSA or corelation length, but it is not always possible, and will change with different observers. 

Time to stop this list but there are many other metrics as well.

### Microstructure Instruments

Now that we know what we're trying to measure (SSA, or correlation length) how do we actually measure? let's talk just about three techniques used in SnowEx 2020 Grand Mesa. 

Micro-computed tomography (microCT) is the only laboratory-based method used here, and it is the gold standard, although it does still come with caveats. The idea of microCT is to remove a sample of snow from a snow pit face, and either cast it with a compound such as diethyl pthalate that is still a liquid at 0° C, or preserve the same at a very cold temperature. Then the sample is sent back to the laboratory, and bombared with x-rays, similar to how you get x-rays to see if a bone is broken at the doctor. For much more on microCT, check out {cite:p}`Heggli2011`. microCT can be used to extract a ton of information about snow microstructure, including SSA, correlation length and many others. 

Integrating spheres are field-based and you make the measurements on samples extracted from the snowpit face. The principle of the measurement is based on firing a laser at the snow sample, within a special reflective hollow sphere, where one side is filled by the snow sample, and measuring how much of the laser is reflected at a sensor at a known geometry. For more information, check out {cite:p}`Gallet2009`.  Most integrating sphere measurements are either made by a commercial firm (A2 Photonics) known as the [IceCube](https://a2photonicsensors.com/icecube-ssa/) or a version constructed at the University of Sherbrooke known as the IRIS {cite:p}`Montpetit2012`. These approaches are set up to measure SSA only. There were three of these at Grand Mesa - one of the Sherbrooke IRIS units, and two IceCubes, one from Finnish Meteorological Institute, and one from Ohio State University.

Snow micropenetrometers are also a field-based approach, but they do not require a snowpit, enabling far more observations to be made. Instead, an automated motor pushes a probe vertically downwards into the snowpack. The probe measures the force required to break snow microstructure, yielding a wealth of information. Snow density, specific surface area and correlation length can be retrieved; for more results see {cite:p}`Lowe2012` and {cite:p}`Proksch2015`. The micropen effort at Grand Mesa was led by Boise State University. 

These methods are not the only ways to measure microstructure! There are several others not mentioned here, but not used at Grand Mesa 2020. Ask if interested.

## SnowEx Microstructure Measurement Data Overview

Of the three methods described above, microCT is by far the most expensive and most time consuming. Samples have to be transported back to the laboratory and the processing time requires a microCT machine. Thus the fewest CT sapmles are taken. 

The integrating spheres require a snowpit to be dug, so we have an intermediate number of them: ~100. 

The micropen measurements are by far the fastest to make, so a cross pattern of SMP measurements was made on orthogonal directions intersecting at the snowpit. There are thousands of SMP profiles from Grand Mesa 2020.

## Working with the data

We're going to do two things! First, we'll intercompare the three different integrating sphere instruments at four different pits where we had multiple instruments operating. We'd expect these data to be fairly self-consistent. Second, we'll compare all three methods (integrating sphere, SMP and microCT) at a single pit where we had all of these measurements present. Here especially with the SMP we would expect to need to intercalibrate the data to match local conditions; so far SSA has only been fit to SMP force measurements in one study, and we should assume we'll need a local calibration to get a tight fit.

### 0. Load needed modules

In [ ]:
# Modules needed to access snowexsql: SnowEx field data database
from snowexsql.db import get_db
from snowexsql.data import LayerData, PointData
from snowexsql.conversions import points_to_geopandas, query_to_geopandas

# Modules needed to work with data
import geoalchemy2.functions as gfunc
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### 1. Intercompare Integrating Sphere Datasets

See Micah's tutorial on datasets for more on this! Won't explain too much here

In [ ]:
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)

# Grab all the equivalent diameter profiles
q = session.query(LayerData).filter(LayerData.type == 'specific_surface_area')
df = query_to_geopandas(q, engine)

# End our database session to avoid hanging transactions
session.close()

Since we want to intercompare integrating spheres, we need to isolate only the sites that actually had multiple integrating spheres measuring the same snow.

In [ ]:
# Grab all the sites with equivalent diameter data (set reduces a list to only its unique entries)
sites = df['site_id'].unique()

# Store all site names that have mulitple SSA instruments
multi_instr_sites = []
instruments = []

for site in sites:

    # Grab all the layers associated to this site
    site_data = df.loc[df['site_id'] == site]

    # Do a set on all the instruments used here
    instruments_used = site_data['instrument'].unique()

    if len(instruments_used) > 1:
        multi_instr_sites.append(site)

# Get a unqique list of SSA instruments that were colocated
instruments = df['instrument'].unique()

Finally, plot all Integrating Sphere SSA profiles at all Multi-Integrating Sphere Sites

In [ ]:
# Setup the subplot for each site for each instrument
fig, axes = plt.subplots(1, len(multi_instr_sites), figsize=(4*len(multi_instr_sites), 8))

# Establish plot colors unique to the instrument
c = ['k', 'm', 'c']
colors = {inst:c[i] for i,inst in enumerate(instruments)}

# Loop over all the multi-instrument sites 
for i, site in enumerate(multi_instr_sites):
    
    # Grab the plot for this site
    ax = axes[i]
    
    # Loop over all the instruments at this site
    for instr in instruments:

        # Grab our profile by site and instrument
        ind = df['site_id'] == site 
        ind2 = df['instrument'] == instr
        profile = df.loc[ind & ind2].copy()

        # Don't plot it unless there is data
        if len(profile.index) > 0:
            
            # Sort by depth so samples that are take out of order won't mess up the plot
            profile = profile.sort_values(by='depth')
            
            # Layer profiles are always stored as strings. 
            profile['value'] = profile['value'].astype(float)
            
            # Plot our profile
            ax.plot(profile['value'], profile['depth'], colors[instr], label=instr)
   
    # Labeling and plot style choices
    ax.legend()
    ax.set_xlabel('SSA [m^2/kg]')
    ax.set_ylabel('Height above snow-soil interface [cm]')
    ax.set_title('Site {}'.format(site.upper()))
    
    # Set the x limits to show more detail
    ax.set_xlim((8, 75))
    
plt.tight_layout()
plt.show()


### 2. Pull the snowmicropenetrometer data and compute SSA

The next step is to grab some SMP data to compare to. We're going to get the SMP at site 2N13, where we have a copule of SSA profiles from interating spheres (as well as microCT data, to be looked at in the next step!). 

There are a few steps here, and one reason for that is just that the SMP data is quite large, and so the full-resolution SMP could not be included in Micah's database. The full resolution profile from SMP is resolved ever 1.25 mm! Instead, the SMP data in Micah's database is sampled to only every 100th datapoint, so it's every 12.5 cm. But the database is still very useful! What we'll do is use the database to find the right profile, then go and download that full resolution dataset from the NSIDC. Easy-peasey!

First up, we'll visualize the location of the SMP profiles, along with the snowpit location.

In [ ]:
site = '2N13'
engine_smp, session_smp = get_db(db_name)
q_smp = session_smp.query(LayerData).filter(LayerData.type == 'force').filter(LayerData.site_id.contains(site) )
df_smp = query_to_geopandas(q_smp, engine_smp)

q_pit=session_smp.query(LayerData).filter(LayerData.type == 'hand_hardness').filter(LayerData.site_id.contains(site) )
df_pit = query_to_geopandas(q_pit, engine_smp)

session_smp.close()

# Plot SMP profile locations with colored by the time they were taken using upside down triangles
ax = df_smp.plot(column='time', cmap='jet', marker='v', label='SMP', figsize=(5,5), markersize=100, edgecolor='black')

ax.plot(df_pit.easting, df_pit.northing, color='black', marker='s', markersize=15, label='Pit ({})'.format(site))

# Add important labels
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')
plt.suptitle('SMP Locations at Site {} Showing Acquisition Order'.format(site), fontsize=16)

# Avoid using Scientific notation for coords.
ax.ticklabel_format(style='plain', useOffset=False)
ax.legend()
# plt.tight_layout()
plt.show()

Next up, let's find the closest SMP profile to the snowpit, and then find the profile ID of that profile, which is in the comments in the database.

In [ ]:
# find closest SMP profile to the pit

# No profile is taken at the same time, so we grab all the unique times and sort them
times = sorted(df_smp['time'].unique())

nprofiles=len(times)

ids=np.empty(nprofiles)

p=0
for t in times:
    ind = df_smp['time'] == t
    data = df_smp.loc[ind].copy()
    ids[p]=data.iloc[0].id
    p+=1
    
i_dists=df_smp['id'].isin(ids)

df_smp_dists=df_smp.loc[i_dists]
df_smp_dists=df_smp_dists.assign(dists=-1)
df_smp_dists['dists']=np.sqrt((df_smp_dists['easting']-df_pit.iloc[0].easting)**2+(df_smp_dists['northing']-df_pit.iloc[0].northing)**2)

    
df_smp_dists.sort_values(by='dists')[['comments','dists']]

So the id of the closest SMP profile is S19M1174.  I went to the [SMP page on NSIDC](https://nsidc.org/data/SNEX20_SMP/versions/1), and went to "Download" and searched for this ID, downloaded the profile, and then re-uploaded to my home directory here in the Jupyter hub. 

Ok next up, we have to compute SSA from the SMP data. For this, we'll use the "snowmiropyn" modules created by the Swiss SLF. You can read more about them [at this site](https://snowmicropyn.readthedocs.io/en/latest/). The software is a little out of date on Python versions; just ignore any warnings that pop up below!

The next cell pulls in the needed modules, and then plots the profile of force measurements needed to break through the snow microstructure.

In [ ]:
from snowmicropyn import Profile
from snowmicropyn import proksch2015

import warnings
warnings.filterwarnings('ignore')

In [ ]:
p = Profile.load('./data/SMP/SNEX20_SMP_S19M1174_2N13_20200206.PNT',)
plt.plot(p.samples.distance, p.samples.force)
# Prettify our plot a bit
plt.title(p.name)
plt.ylabel('Force [N]')
plt.xlabel('Depth [mm]')
plt.show()

The next step is the actual calculation of SSA from the force data. It then displays the data and lets you see that there is now a column called SSA!

In [ ]:
p2015 = proksch2015.calc(p.samples,window=10) 
p2015

### 3. Read microCT data, and compare intergrating sphere, SMP and CT data

This module is a custom file I wrote to read in the microCT datafiles which are stored as text. I'm a Python newbie so don't laugh at my poor coding!

Equivalent grain size is a useful quantity to compare: because it's proportional to 1/SSA, and because after a point as you increase SSA more and more, all fine-grained snow acts more-or-less the same (converging to e.g. the "fine-grained" curve in Figure 1, above), we'll look at equivalent diameter instead of SSA in this comparison.

In [ ]:
from read_CT_txt_files import read_CT_txt_files

In [ ]:
# read micro CT for 2N13
data_dir='./data/microCT/txt/'
[SSA_CT,height_min,height_max]=read_CT_txt_files(data_dir)

In [ ]:
# get data integrating sphere data for 2N13 and plot it 
site='2N13'
engine_is, session_is = get_db(db_name)
q_is = session_is.query(LayerData).filter(LayerData.type == 'specific_surface_area').filter(LayerData.site_id.contains(site) )
df_is = query_to_geopandas(q_is, engine_is)
instruments_site = df_is['instrument'].unique()

# Loop over all the integrating sphere instruments at this site. plot equivalent diameter
fig,ax = plt.subplots()
for instr in instruments_site:

    # Grab our profile by site and instrument
    ind = df['site_id'] == site 
    ind2 = df['instrument'] == instr
    profile = df.loc[ind & ind2].copy()

    # Don't plot it unless there is data
    if len(profile.index) > 0:

        # Sort by depth so samples that are take out of order won't mess up the plot
        profile = profile.sort_values(by='depth')

        # Layer profiles are always stored as strings. 
        profile['value'] = 6/917/profile['value'].astype(float)*1000

        # Plot our profile
        ax.plot(profile['value'], profile['depth'], colors[instr], label=instr)
        
#All that's left to do is plot the CT and the SMP and label the plot!
ax.plot(6/917/SSA_CT*1000,height_min,label='microCT')        #CT data

ax.plot(6/917/p2015.P2015_ssa*1000,(max(p2015.distance)-p2015.distance)/10,label='SMP') #SMP data

# Labeling and plot style choices
ax.legend()
ax.set_xlabel('Equivalent diameter, mm')
ax.set_ylabel('Height above snow-soil interface [cm]')
ax.set_title('Site {}'.format(site.upper()))
    
plt.tight_layout()
plt.show()